In [13]:
import json
import logging
from jinja2 import Template


def gen_tensorflow_client_string(generated_tensor_data, model_name):
    """
      Generate TensorFlow SDK in Python.
      Args:
        generated_tensor_data: Example is {"keys": [[1.0], [2.0]], "features": [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}
    """
    code_template = """#!/usr/bin/env 
    python import requests
    def main():
        endpoint = "http://127.0.0.1:8500"
        json_data = {"model_name": "{{ model_name }}", "data": {{ tensor_data }} }
        result = requests.post(endpoint, json=json_data)
        print(result.json())
    if __name__ == "__main__":
        main()
    """

    generated_tensor_data_string = json.dumps(generated_tensor_data)
    template = Template(code_template)
    generate_code = template.render(
        model_name=model_name, tensor_data=generated_tensor_data_string)
    logging.debug("Generate the code in Python:\n{}".format(generate_code))

    return generate_code


def gen_tensorflow_client(generated_tensor_data, model_name):
    generate_code = gen_tensorflow_client_string(generated_tensor_data, model_name)
    generated_code_filename = "client.py"
    with open(generated_code_filename, "w") as f:
        f.write(generate_code)
    logging.info('Save the generated code in {}, try "python {}"'.format(
        generated_code_filename, generated_code_filename))

In [14]:
print gen_tensorflow_client_string({'x':1}, 'model_name')

#!/usr/bin/env 
    python import requests
    def main():
        endpoint = "http://127.0.0.1:8500"
        json_data = {"model_name": "model_name", "data": {"x": 1} }
        result = requests.post(endpoint, json=json_data)
        print(result.json())
    if __name__ == "__main__":
        main()
    


In [9]:
gen_tensorflow_client({'x':1}, 'model_name')

In [ ]:
    def push_directory(self):
        with tempfile.TemporaryDirectory() as tmpdir:
            archive = self.archive(rootdir=".", output_dir=tmpdir)
            remote.put(archive, "/tmp/rorolite-project.tgz")

            with lcd(tmpdir):
                self.generate_supervisor_config(rootdir=tmpdir)

                supervisor_archive = self.archive(tmpdir, base_dir=".rorolite", filename="rorolite-supervisor")
                remote.put(supervisor_archive, "/tmp/rorolite-supervisor.tgz")

            with remote.cd(self.deploy_root):
                remote.sudo("chown {} .".format(env.user))
                remote.run("tar xzf /tmp/rorolite-project.tgz")
                remote.run("tar xzf /tmp/rorolite-supervisor.tgz")

In [58]:
import shutil,os,sys
PY2 = (sys.version_info.major == 2)
if PY2:
    from backports import tempfile
else:
    import tempfile

class deploy(object):
    def deploy(self, async=False):
        print("Deploying project {}. This may take a few moments ...".format(self.name))
        with tempfile.TemporaryDirectory() as tmpdir:
            archive = self.archive(tmpdir)
            size = os.path.getsize(archive)
            with open(archive, 'rb') as f:
                format = 'tar'
                response =  self.client.deploy(
                    project=self.name,
                    archived_project=f,
                    size=size,
                    format=format,
                    async=async
                )
            if async:
                return Task(response['task_id'], self.SERVER_URL)
            else:
                return response
    def archive(self, rootdir, output_dir=None, format='gztar', base_dir=".", filename='arthur-project'):
        output_dir = output_dir or rootdir
        base_name = os.path.join(output_dir, filename)
        return shutil.make_archive(base_name, format, root_dir=rootdir, base_dir=base_dir)
x=deploy()
y=x.archive('./',filename='client.py')

IndentationError: expected an indented block (<ipython-input-58-135e6f67b968>, line 26)

In [30]:
# 管道执行命令， 命令小输出还可以， 如果输出比较大推荐走文件输出流
def exec_cmd2(cmd):
    
    #Log.debug("cmd: %s" %(cmd))
    content = ""
    try:
        p = Popen(cmd, bufsize = 4096, stdout = PIPE, shell = True)
        
        while True:
            cont = p.stdout.read()
            if cont == "": break
            content += cont
            #Log.debug("contLen: %d" %(len(cont)))
            time.sleep(1)
        retState = p.wait()
        
        return retState, content
    except Exception, e:
        #Log.err("(%s)" %(traceback.format_exc()))
        return 255, "cmd(%s) err: %s" %(str(cmd), str(e))
from subprocess import * 
import time
cmd="ls /tmp/ 2>/dev/null || echo"
_,output=exec_cmd2(cmd)
versions = [int(v) for v in output.strip().split('\n') if v.isnumeric()]

AttributeError: 'str' object has no attribute 'isnumeric'

In [32]:
import fabric.api as remote
output = remote.run("ls /tmp/ 2>/dev/null || echo", quiet=True)

NetworkError: Name lookup failed for d

In [31]:
output.strip().split('\n')

['1',
 'com.apple.launchd.HDUlEZN3U1',
 'com.apple.launchd.mtVJBaiFwJ',
 'mongodb-27017.sock',
 'mysql.sock',
 'mysql.sock.lock',
 'powerlog',
 'signin_eventlist-CDP - Handle CDP enroll.plist']

In [25]:
output

(0,
 'com.apple.launchd.HDUlEZN3U1\ncom.apple.launchd.mtVJBaiFwJ\nmongodb-27017.sock\nmysql.sock\nmysql.sock.lock\npowerlog\nsignin_eventlist-CDP - Handle CDP enroll.plist\n')

In [23]:
class ShellCommandException(Exception):
    pass

import subprocess
def exec_cmd(cmd, throw_on_error=True, env=None, stream_output=False, cwd=None, cmd_stdin=None,
             **kwargs):
    """
    Runs a command as a child process.
    A convenience wrapper for running a command from a Python script.
    Keyword arguments:
    cmd -- the command to run, as a list of strings
    throw_on_error -- if true, raises an Exception if the exit code of the program is nonzero
    env -- additional environment variables to be defined when running the child process
    cwd -- working directory for child process
    stream_output -- if true, does not capture standard output and error; if false, captures these
      streams and returns them
    cmd_stdin -- if specified, passes the specified string as stdin to the child process.
    Note on the return value: If stream_output is true, then only the exit code is returned. If
    stream_output is false, then a tuple of the exit code, standard output and standard error is
    returned.
    """
    cmd_env = os.environ.copy()
    if env:
        cmd_env.update(env)

    if stream_output:
        child = subprocess.Popen(cmd, env=cmd_env, cwd=cwd, universal_newlines=True,
                                 stdin=subprocess.PIPE, **kwargs)
        child.communicate(cmd_stdin)
        exit_code = child.wait()
        if throw_on_error and exit_code is not 0:
            raise ShellCommandException("Non-zero exitcode: %s" % (exit_code))
        return exit_code
    else:
        child = subprocess.Popen(
            cmd, env=cmd_env, stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE,
            cwd=cwd, universal_newlines=True, **kwargs)
        (stdout, stderr) = child.communicate(cmd_stdin)
        exit_code = child.wait()
        if throw_on_error and exit_code is not 0:
            raise ShellCommandException("Non-zero exit code: %s\n\nSTDOUT:\n%s\n\nSTDERR:%s" %
                                        (exit_code, stdout, stderr))
        return exit_code, stdout, stderr
cmd="ls /tmp/ 2>/dev/null || echo"
output=exec_cmd(cmd)

OSError: [Errno 2] No such file or directory

In [37]:
%%file get_filename.py
import sys,os
def get_filename(backstep=0):
    """
    Get the file name of the current code line.
    :param backstep:
        will go backward (one layer) from the current function call stack
    """
    return os.path.basename(
        sys._getframe(backstep + 1).f_code.co_filename)  # pylint:disable=W0212
print get_filename()

Overwriting get_filename.py


In [42]:
from cup import decorators
import cup
def is_port_used(port):
    """
    judge if the port is used or not (It's not 100% sure as next second, some
    other process may steal the port as soon after this function returns)
    :platform:
        linux only (netstat command used inside)
    :param port:
        expected port
    :return:
        return True if the port is used, False otherwise
    """
    #@decorators.needlinux
    def __is_port_used(port):
        """internal func"""
        cmd = "netstat -nl | grep ':%s '" % (port)
        ret = cup.shell.ShellExec().run(cmd, 10)
        if 0 != ret['returncode']:
            return False
        stdout = ret['stdout'].strip()
        if 0 == len(stdout):
            return False
        else:
            return True
    return __is_port_used(port)
is_port_used(9011)

False

In [48]:
import socket 
def get_host_ip(): 
    """
    get host ip address
    获取本机IP地址

    :return:
    """ 
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 
    try: 
        s.connect(('8.8.8.8', 80)) 
        ip = s.getsockname()[0] 
    finally: 
        s.close() 
    return ip 
def is_port_used(ip, port): 
    """
    check whether the port is used by other program
    检测端口是否被占用

    :param ip:
    :param port:
    :return:
    """ 
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
    try: 
        s.connect((ip, port))
        return True
    except OSError: 
        return False 
    finally: 
        s.close()
    # 测试 
if __name__ == '__main__':
    host_ip = get_host_ip() 
    print(host_ip) 
    print(is_port_used(host_ip, 80))


192.168.100.245


error: [Errno 61] Connection refused

In [51]:
import random
def execshell_withpipe_ex(cmd, b_printcmd=True):
    """
    Deprecated. Recommand using ShellExec.
    """
    strfile = '/tmp/%s.%d.%d' % (
        'shell_env.py', int(os.getpid()), random.randint(100000, 999999)
    )
    os.mknod(strfile)
    cmd = cmd + ' 1>' + strfile + ' 2>/dev/null'
    os.system(cmd)
    if True == b_printcmd:
        print cmd
    fphandle = open(strfile, 'r')
    lines = fphandle.readlines()
    fphandle.close()
    os.unlink(strfile)
    return lines
def execshell_withpipe_str(cmd, b_printcmd=True):
    """
    Deprecated. Recommand using ShellExec.
    """
    return ''.join(execshell_withpipe_ex(cmd, b_printcmd))
def is_proc_alive(procname, is_whole_word=False, is_server_tag=False, filters=False):
    """
    Deprecated. Recommand using cup.oper.is_proc_exist
    """
    # print procName
    if is_whole_word:
        cmd = "ps -ef|grep -w '%s'$ |grep -v grep" % procname
    else:
        cmd = "ps -ef|grep -w '%s' |grep -v grep" % procname

    if is_server_tag:
        cmd += '|grep -vwE "vim |less |vi |tail |cat |more "'
    if filters:
        if isinstance(filters, str):
            cmd += "|grep -v '%s'" % filters
        elif isinstance(filters, list):
            for _, task in enumerate(filters):
                cmd += "|grep -v '%s'" % task
    cmd += '|wc -l'
    rev = execshell_withpipe_str(cmd, False)
    if int(rev) > 0:
        return True
    else:
        return False
is_proc_alive('web_main.py')

OSError: [Errno 1] Operation not permitted

In [56]:
def get_pid(process_path, grep_string):
    """
    will return immediately after find the pid which matches
    1. ps -ef|grep %s|grep -v grep|grep -vE "^[vim|less|vi|tail|cat|more] "
    '|awk '{print $2}'
    2. workdir is the same as ${process_path}
    :param process_path:
        process that runs on
    :param grep_string:
        ps -ef|grep ${grep_string}
    :return:
        return None if not found. Otherwise, return the pid
    """
    cmd = (
        'ps -ef|grep \'%s\'|grep -v grep|grep -vwE "vim |less |vi |tail |cat |more "'
        '|awk \'{print $2}\''
    ) % (grep_string)
    ret = cup.shell.ShellExec().run(cmd, 10)
    pids = ret['stdout'].strip().split('\n')
    if len(pids) == 0 or len(pids) == 1 and len(pids[0]) == 0:
        return None
    for pid in pids:
        for sel_path in ["cwd", "exe"]:
            cmd = 'ls -l /proc/%s/%s|awk \'{print $11}\' ' % (pid, sel_path)
            ret = cup.shell.ShellExec().run(cmd, 10)
            pid_path = ret['stdout'].strip().strip()
            if pid_path.find(process_path) == 0:
                return pid
    return None

# end linux functionalities }}
# vi:set tw=0 ts=4 sw=4 nowrap fdm=indent
p='/Users/leepand/Downloads/recom/python_web/Arthur_deploy_web/src/web'
g='web_main'
print get_pid(p,g)

None
